In [1]:
import wandb
import torch
import gym

import tensorflow as tf
import torch.nn as nn
import numpy as np
import torch.optim as optim
from collections import deque
from torch.distributions import Beta
from torch.utils.data.sampler import BatchSampler, SubsetRandomSampler
import torch.nn.functional as F

print('TF with GPU: ' + str(tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None)))
print('Torch with GPU: {}'.format(torch.cuda.is_available()))
wandb.login()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
TF with GPU: True
Torch with GPU: True


wandb: Currently logged in as: camperko (use `wandb login --relogin` to force relogin)


In [2]:
config = {
    "num_episodes": 1500,
    "num_epochs": 2000,
    "image_batch": 4,
    "lr": 1e-3,
    "logging_interval": 5,
    "render": True,
    "action_count": 4,
    "memory_cap": 2560,
    "ppo_batch": 32,
    "ppo_epoch": 10,
    "loss_clip": 0.1,
    "gamma_discount": 0.99,
    "render_interval": 50
}

action_type = np.dtype([
    ('last_state', np.float64, (config['image_batch'], 96, 96)),
    ('curr_state', np.float64, (config['image_batch'], 96, 96)),
    ('act', np.float64, (3,)),
    ('act_prob', np.float64),
    ('reward', np.float64)
])

def rgb2gray(rgb):
    return np.dot(rgb[..., :], [0.299, 0.587, 0.114]) / 128. - 1.

In [3]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(config['image_batch'], 8, kernel_size=(4, 4), stride=(2,2)),
            nn.ReLU(),
            nn.Conv2d(8, 16, kernel_size=(3, 3), stride=(2,2)),
            nn.ReLU(),
            nn.Conv2d(16, 32, kernel_size=(3, 3), stride=(2,2)),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=(3, 3), stride=(2,2)),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=(3, 3), stride=(1,1)),
            nn.ReLU(),
            nn.Conv2d(128, 256, kernel_size=(3, 3), stride=(1,1)),
            nn.ReLU(),
        )
        self.v = nn.Sequential(
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )
        self.fc = nn.Sequential(
            nn.Linear(256, 64),
            nn.ReLU()
        )
        self.alpha_head = nn.Sequential(
            nn.Linear(64, 3),
            nn.Softplus()
        )
        self.beta_head = nn.Sequential(
            nn.Linear(64, 3),
            nn.Softplus()
        )
        self.apply(self._weights_init)

    @staticmethod
    def _weights_init(m):
        if isinstance(m, nn.Conv2d):
            nn.init.xavier_uniform_(m.weight, gain=nn.init.calculate_gain('relu'))
            nn.init.constant_(m.bias, 0.1)

    def forward(self, x):
        x = self.cnn(x).view(-1, 256)
        v = self.v(x)
        x = self.fc(x)
        alpha = self.alpha_head(x) + 1
        beta = self.beta_head(x) + 1

        return (alpha, beta), v

In [5]:
class Agent():

    def __init__(self):
        self.net = Net().double().to(device)
        self.optimizer = optim.Adam(self.net.parameters(), lr=config['lr'])

        self.memory = np.empty(config['memory_cap'], dtype=action_type)
        self.memory_index = 0

    def choose_action(self, curr_state):
        with torch.no_grad():
            action, _ = self.net(torch.from_numpy(curr_state).double().to(device).unsqueeze(0))
        distribution = Beta(action[0], action[1])
        act = distribution.sample()
        act_prob = distribution.log_prob(act).sum(dim=1)
        return act.squeeze().cpu().numpy(), act_prob.item()

    def insert_into_memory(self, memory_sample):
        self.memory[self.memory_index] = memory_sample
        self.memory_index += 1
        if self.memory_index == config['memory_cap']:
            self.memory_index = 0

    def ready_for_update(self):
        return True if self.memory_index == 0 else False

    def update_self(self):
        last_state = torch.tensor(self.memory['last_state'], dtype=torch.double).to(device)
        curr_state = torch.tensor(self.memory['curr_state'], dtype=torch.double).to(device)
        action = torch.tensor(self.memory['act'], dtype=torch.double).to(device)
        rew = torch.tensor(self.memory['reward'], dtype=torch.double).to(device).view(-1, 1)
        action_prob = torch.tensor(self.memory['act_prob'], dtype=torch.double).to(device).view(-1, 1)

        with torch.no_grad():
            target_v = rew + config['gamma_discount'] * self.net(curr_state)[1]
            adv = target_v - self.net(last_state)[1]

        for _ in range(config['ppo_epoch']):
            for index in BatchSampler(SubsetRandomSampler(range(config['memory_cap'])), config['ppo_batch'], False):
                alpha, beta = self.net(last_state[index])[0]
                dist = Beta(alpha, beta)
                a_logp = dist.log_prob(action[index]).sum(dim=1, keepdim=True)
                ratio = torch.exp(a_logp - action_prob[index])

                surr1 = ratio * adv[index]
                surr2 = torch.clamp(ratio, 1.0 - config['loss_clip'], 1.0 + config['loss_clip']) * adv[index]
                action_loss = -torch.min(surr1, surr2).mean()
                value_loss = F.smooth_l1_loss(self.net(last_state[index])[1], target_v[index])
                loss = action_loss + 2. * value_loss

                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()

In [ ]:
env = gym.make('CarRacing-v0')
reward_threshold = env.spec.reward_threshold
env.close()

agent = Agent()

run = wandb.init(project="car-racing")

wandb.config.update(config)

total_rewards = deque(maxlen=100)
reward_mean = 0

for i_episode in range(config['num_episodes']):
    env = gym.make('CarRacing-v0')
    observation = env.reset()
    old_state = [rgb2gray(observation)] * config['image_batch']
    new_state = [rgb2gray(observation)] * config['image_batch']
    done = False
    total_reward = 0
    reward_history = deque(maxlen=100)

    for _ in range(config['num_epochs']):
        if config['render'] and i_episode % config['render_interval'] == 0 and i_episode != 0:
            env.render()

        todo_action, todo_action_prob = agent.choose_action(np.array(new_state))

        todo_action = todo_action * np.array([2., 1., 1.]) + np.array([-1., 0., 0.])

        inner_reward = 0
        for i in range(config['action_count']):
            observation, reward, done, _ = env.step(todo_action)

            if np.mean(observation[:, :, 1]) > 185.0:
                reward -= 0.05

            reward += 100 if done else 0

            inner_reward += reward

            reward_history.append(reward)
            if done or np.mean(reward_history) <= -0.1:
                break

        new_state.pop(0)
        new_state.append(rgb2gray(observation))

        agent.insert_into_memory((np.array(old_state), np.array(new_state), todo_action, todo_action_prob, inner_reward))
        if agent.ready_for_update():
            print('Updating agent')
            agent.update_self()

        old_state.pop(0)
        old_state.append(rgb2gray(observation))

        total_reward += inner_reward

        if done:
            break

    total_rewards.append(total_reward)
    wandb.log({"episode_reward": total_reward}, commit=True)
    reward_mean = np.mean(total_rewards)
    if i_episode % config['logging_interval'] == 0:
        print('Total reward for episode {} is {}'.format(i_episode, total_reward))
    if reward_mean >= reward_threshold:
        print('Env solved in {} episodes'.format(i_episode))
        break

    env.close()

if reward_mean < reward_threshold:
    print('Env not solved in {} episodes'.format(config['num_episodes']))
    print('Mean of score for last 100 runs is {}'.format(reward_mean))


run.finish()


C:\Programming\NSIETE\Assignment\NSIETE-assignment_3\venv\lib\site-packages\gym\logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
wandb: wandb version 0.10.30 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Track generation: 1107..1388 -> 281-tiles track
Total reward for episode 0 is 78.57142857142861
Track generation: 991..1250 -> 259-tiles track
Track generation: 1035..1302 -> 267-tiles track
Track generation: 1236..1549 -> 313-tiles track
Track generation: 1048..1314 -> 266-tiles track
Track generation: 1176..1474 -> 298-tiles track
Total reward for episode 5 is 74.07407407407405
Track generation: 969..1220 -> 251-tiles track
Track generation: 1209..1515 -> 306-tiles track
Track generation: 1075..1348 -> 273-tiles track
Track generation: 1160..1454 -> 294-tiles track
Updating agent


In [29]:
agent2 = Agent()

env = gym.make('CarRacing-v0')

observation = env.reset()

old_state = [rgb2gray(observation)] * config['image_batch']
new_state = [rgb2gray(observation)] * config['image_batch']
done = False
for i in range(5):
    env.render()
    todo_action, todo_action_prob = agent2.choose_action(np.asarray(new_state))

    for i in range(config['action_count']):
        observation, reward, done, _ = env.step(todo_action)

        if i != config['action_count'] - 1:
            del observation

        if done:
            break

    new_state.pop(0)
    new_state.append(rgb2gray(observation))

    old_state.pop(0)
    old_state.append(rgb2gray(observation))

    # del observation
    if done:
        break
env.close()

C:\Programming\NSIETE\Assignment\NSIETE-assignment_3\venv\lib\site-packages\gym\logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


Track generation: 1236..1549 -> 313-tiles track
Action from net
(tensor([[1.7263, 1.7781, 1.7562]], device='cuda:0', dtype=torch.float64), tensor([[1.7590, 1.7411, 1.6706]], device='cuda:0', dtype=torch.float64))
Distribution from action
Beta()
Sample action
tensor([[0.7013, 0.7197, 0.5666]], device='cuda:0', dtype=torch.float64)
Returned action
[0.70132948 0.71973825 0.56657877]
Action from net
(tensor([[1.7381, 1.7578, 1.7591]], device='cuda:0', dtype=torch.float64), tensor([[1.7647, 1.7489, 1.6763]], device='cuda:0', dtype=torch.float64))
Distribution from action
Beta()
Sample action
tensor([[0.2835, 0.4353, 0.6308]], device='cuda:0', dtype=torch.float64)
Returned action
[0.28346383 0.43531157 0.63079592]
Action from net
(tensor([[1.7155, 1.7353, 1.7545]], device='cuda:0', dtype=torch.float64), tensor([[1.7564, 1.7567, 1.6804]], device='cuda:0', dtype=torch.float64))
Distribution from action
Beta()
Sample action
tensor([[0.8580, 0.6637, 0.3153]], device='cuda:0', dtype=torch.float64

In [30]:
env = gym.make('CarRacing-v0')

print(env.action_space)

env.close()

Box(-1.0, 1.0, (3,), float32)


C:\Programming\NSIETE\Assignment\NSIETE-assignment_3\venv\lib\site-packages\gym\logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
